In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import os
from sklearn.preprocessing import OneHotEncoder
import nltk

# Extracting recipes from csv

In [78]:
#users = pd.read_csv('../Data/PP_users.csv')
#recipes = pd.read_csv('../Data/PP_recipes.csv')
raw_interactions = pd.read_csv('./RAW_interactions.csv')
raw_recipes = pd.read_csv('./RAW_recipes.csv')

In [79]:
print(raw_interactions.shape)
raw_interactions.head()

(1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [80]:
print(raw_recipes.shape)
raw_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [81]:
# merge recipes and interactions tables

dfinal = raw_interactions.merge(raw_recipes, how='inner', left_on='recipe_id', right_on='id')
print(dfinal.shape)
dfinal.head()

(1132367, 17)


,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


# Change date format (from str to date format)

In [82]:
from datetime import datetime

dfinal['date'] = dfinal['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [83]:
print(type(dfinal['date'].iloc[0]))

<class 'datetime.date'>


# Categorising of recipes
Using tags

In [84]:
def str_to_list(row):

    x = row[1:-1] # remove []
    x = x.split(',')
    x = [x.strip()[1:-1] for x in x]
        
    return x

In [85]:
df_tags = dfinal[['recipe_id','tags']]
df_tags.drop_duplicates(inplace=True)
df_tags.reset_index(drop=True,inplace=True) # reset index
df_tags.head()

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags
0,40893,"['weeknight', 'time-to-make', 'course', 'main-..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i..."


In [86]:
df_tags['tag_list'] = df_tags['tags'].apply(lambda x: str_to_list(x))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
df_tags.head()

,recipe_id,tags,tag_list
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred..."


### Find unique tags

In [88]:
all_tags = list()
for tags in df_tags['tag_list']:
    for t in tags:
        if t not in all_tags:
            print(t)
            all_tags.append(t)

weeknight
time-to-make
course
main-ingredient
preparation
occasion
soups-stews
beans
vegetables
easy
crock-pot-slow-cooker
dietary
equipment
30-minutes-or-less
for-large-groups
5-ingredients-or-less
drop-cookies
desserts
lunch
oven
potluck
fall
finger-food
heirloom-historical
holiday-event
kid-friendly
picnic
spring
summer
winter
cakes
cookies-and-brownies
easter
chocolate
gifts
christmas
hanukkah
new-years
seasonal
comfort-food
independence-day
ramadan
rosh-hashana
valentines-day
taste-mood
to-go
number-of-servings
presentation
15-minutes-or-less
condiments-etc
inexpensive
3-steps-or-less
cuisine
north-american
low-protein
healthy
jams-and-preserves
fruit
american
low-fat
low-sodium
low-cholesterol
low-saturated-fat
healthy-2
northeastern-united-states
low-in-something
berries
raspberries
60-minutes-or-less
poultry
chicken
high-protein
high-in-something
meat
main-dish
beef
mexican
low-carb
lamb-sheep
african
moroccan
stove-top
spicy
4-hours-or-less
for-1-or-2
casseroles
southwestern-u

In [89]:
# Selected category
category = ['meat', 'seafood', 'salads', 'vegetables', 'soup', 'pasta', 'desserts', 'breads', 'burgers', 'beverages']

In [90]:
# Creating new column
df_tags['category'] = ""
df_tags

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [91]:
for index, row in df_tags.iterrows():
    
    # if the recipe falls under multiple categories, use the first category found in the tag
    if df_tags.at[index, 'category'] == "": 
        
        for cat in category: 
            if cat in row['tags']:
                df_tags.at[index, 'category'] = cat    

In [92]:
df_tags

,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",meat
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [93]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
              37321
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

### Drop recipes that do not fall in any category

In [94]:
df_tags = df_tags[df_tags['category'] != ""]
df_tags


,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231629,492861,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, cui...",beverages
231630,273409,"['60-minutes-or-less', 'time-to-make', 'course...","[60-minutes-or-less, time-to-make, course, mai...",vegetables
231631,249924,"['time-to-make', 'course', 'main-ingredient', ...","[time-to-make, course, main-ingredient, cuisin...",meat
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood


In [95]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

In [97]:
df_tags.drop(columns = ['tags', 'tag_list'], inplace=True)

In [98]:
df_tags

,recipe_id,category
0,40893,soup
1,44394,desserts
2,85009,vegetables
3,120345,desserts
4,134728,meat
...,...,...
231629,492861,beverages
231630,273409,vegetables
231631,249924,meat
231632,470995,seafood


# Extract nutritional Components

- from string to respective categories
- convert them to float type

In [99]:
df_nutri = dfinal[['recipe_id', 'name','n_ingredients','minutes','nutrition']]
df_nutri.drop_duplicates(inplace=True)
df_nutri.reset_index(drop=True,inplace=True)
df_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,name,n_ingredients,minutes,nutrition
0,40893,white bean green chile pepper soup,9,495,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]"
1,44394,devilicious cookie cake delights,4,20,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]"
2,85009,baked potato toppings,13,10,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]"
3,120345,sugared raspberries,2,10,"[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]"
4,134728,kfc honey bbq strips,12,40,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]"
...,...,...,...,...,...
231632,470995,perfectly fried shrimps with garlic,3,8,"[397.0, 62.0, 0.0, 8.0, 10.0, 28.0, 1.0]"
231633,241491,maple apple baked beans,10,375,"[518.1, 29.0, 138.0, 26.0, 34.0, 35.0, 24.0]"
231634,257796,slow cooker garlic chicken with rosemary,9,440,"[566.2, 59.0, 12.0, 15.0, 86.0, 55.0, 3.0]"
231635,72730,cranberry peach maple relish,6,22,"[1941.7, 3.0, 1569.0, 3.0, 10.0, 1.0, 154.0]"


In [100]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri.nutrition.str.split(",",expand=True)

In [101]:
df_nutri['calories'] =  df_nutri['calories'].apply(lambda x: x.replace('[','')) 
df_nutri['carbohydrates (PDV)'] =  df_nutri['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [102]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

In [103]:
print(df_nutri.shape)
df_nutri.drop(['nutrition'], axis=1,inplace=True)
df_nutri.head()

(231637, 12)


,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0
1,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0
2,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0
3,120345,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0
4,134728,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,4.0,10.0


# Combine dataset 
Merge with user information

In [104]:
dfuser = dfinal[['user_id', 'date', 'review','rating','recipe_id']]
dfuser.drop_duplicates(inplace=True)
dfuser.reset_index(drop=True,inplace=True)
dfuser

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,date,review,rating,recipe_id
0,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,40893
1,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,40893
2,8937,2002-12-01,This worked very well and is EASY. I used not...,4,44394
3,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,85009
4,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,85009
...,...,...,...,...,...
1132362,417131,2007-09-05,I made this over the weekend and changed it up...,4,241491
1132363,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5,241491
1132364,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4,257796
1132365,116593,2003-12-09,Another approach is to start making sauce with...,0,72730


In [105]:
df_combine = df_nutri.merge(dfuser, how='inner', on='recipe_id')
df_combine

,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4
1,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5
2,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0,8937,2002-12-01,This worked very well and is EASY. I used not...,4
3,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5
4,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132362,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,24.0,417131,2007-09-05,I made this over the weekend and changed it up...,4
1132363,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,24.0,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5
1132364,257796,slow cooker garlic chicken with rosemary,9,440,566.2,59.0,12.0,15.0,86.0,55.0,3.0,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4
1132365,72730,cranberry peach maple relish,6,22,1941.7,3.0,1569.0,3.0,10.0,1.0,154.0,116593,2003-12-09,Another approach is to start making sauce with...,0


In [106]:
df_combine = df_tags.merge(df_combine, how='inner', on='recipe_id')
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5
2,44394,desserts,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0,8937,2002-12-01,This worked very well and is EASY. I used not...,4
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,39.0,8.0,226863,2009-06-19,"I made this without the bread, and added some ...",5
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,39.0,8.0,39835,2010-05-15,This chicken was great (skipped the bread). My...,5
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,39.0,8.0,496803,2011-08-21,I really loved this! I cut the recipe back to...,5
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,28.0,1.0,169430,2013-04-17,Just as the title promised this is a lovely sh...,5


### Filtering data
- User that has reviewed 500 <= reviews <= 1000 reviews

In [304]:
user_filter = df_combine.groupby("user_id").filter(lambda x: 500 <= len(x)  <= 1000)

In [305]:
user_filter

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4
16,200236,meat,lamb stew with tomatoes chickpeas and spices,16,150,606.5,65.0,12.0,34.0,65.0,83.0,7.0,68960,2006-12-19,We really enjoyed this and so did both my youn...,4
21,225241,vegetables,chicken tamale pie for 2 ww core,16,45,248.0,4.0,28.0,21.0,44.0,3.0,11.0,353579,2007-08-14,"Very good, made with ingredients I usually hav...",5
35,79222,soup,potato crab chowder,13,45,274.2,12.0,15.0,18.0,32.0,22.0,12.0,446143,2008-02-01,"Oh, how wonderful! I doubled the crab, and ad...",4
57,321038,desserts,hg s grab n go breakfast cookies weight wat...,14,22,182.4,2.0,50.0,7.0,11.0,1.0,13.0,539686,2011-03-28,My very health conscious daughter and I triple...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978869,286499,meat,salsa grilled chicken,4,60,345.4,11.0,25.0,74.0,108.0,7.0,5.0,844554,2012-06-06,Very good recipe! I did have to use salsa verd...,5
978871,40514,vegetables,sun dried tomato bruschetta with goat cheese,5,15,94.6,5.0,4.0,8.0,8.0,10.0,3.0,469903,2008-06-24,Simple flavors of summertime! Instead of bread...,4
978882,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,6.0,1.0,145352,2007-05-14,Oh I loved this! So quick and so tasty! I made...,5
978886,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,6.0,1.0,333017,2009-06-24,"This was just OK. When I first tasted it, I fo...",3


### Sort dataframe by date

In [308]:
final = user_filter.sort_values(by="date")
final.drop_duplicates(inplace=True)
print(final.shape)
final.head()

(90333, 16)


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
743864,5315,beverages,icy holiday punch,6,1440,54.5,0.0,50.0,0.0,0.0,0.0,4.0,8629,2001-04-24,I brought this to an office holiday party and ...,4
192089,8661,salads,oliveau salad dressing,8,20,168.7,28.0,3.0,3.0,0.0,12.0,0.0,13483,2001-08-05,Good recipe and easy to make. I found it a li...,4
168111,10859,vegetables,broccoli cauliflower and rice casserole,5,85,228.8,24.0,12.0,46.0,18.0,32.0,4.0,8629,2001-08-15,This is a nice casserole. It is very easy to ...,2
449496,902,vegetables,microwaved garlic mushroom rolls,7,0,1188.6,88.0,53.0,69.0,75.0,153.0,46.0,13483,2001-08-16,Very easy to make for a lunch. I served it co...,3
772482,10722,desserts,cherry cream cheese pie,6,1455,317.1,32.0,29.0,4.0,6.0,65.0,9.0,13483,2001-09-04,I like easy and good. I bought the pie shell ...,5


In [309]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90333 entries, 743864 to 246716
Data columns (total 16 columns):
recipe_id              90333 non-null int64
category               90333 non-null object
name                   90333 non-null object
n_ingredients          90333 non-null int64
minutes                90333 non-null int64
calories               90333 non-null float64
total fat (PDV)        90333 non-null float64
sugar (PDV)            90333 non-null float64
sodium (PDV)           90333 non-null float64
protein (PDV)          90333 non-null float64
saturated fat (PDV)    90333 non-null float64
carbohydrates (PDV)    90333 non-null float64
user_id                90333 non-null int64
date                   90333 non-null object
review                 90333 non-null object
rating                 90333 non-null int64
dtypes: float64(7), int64(5), object(4)
memory usage: 11.7+ MB


In [310]:
# Convert to csv
final.to_csv('./Dataset/final_dataset.csv', index=False)

# Train & Test Dataset

In [6]:
all_dataset = pd.read_csv('./Dataset/final_dataset_log.csv')
all_dataset.drop('Unnamed: 0', axis = 1, inplace=True)
all_dataset.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,5315,beverages,icy holiday punch,6,1440,54.5,1.0,50.0,1.0,1.0,...,2001-04-24,I brought this to an office holiday party and ...,4,3.998201,0.000000,3.912023,0.000000,0.000000,0.000000,1.386294
1,8661,salads,oliveau salad dressing,8,20,168.7,28.0,3.0,3.0,1.0,...,2001-08-05,Good recipe and easy to make. I found it a li...,4,5.128122,3.332205,1.098612,1.098612,0.000000,2.484907,0.000000
2,10859,vegetables,broccoli cauliflower and rice casserole,5,85,228.8,24.0,12.0,46.0,18.0,...,2001-08-15,This is a nice casserole. It is very easy to ...,2,5.432848,3.178054,2.484907,3.828641,2.890372,3.465736,1.386294
3,902,vegetables,microwaved garlic mushroom rolls,7,0,1188.6,88.0,53.0,69.0,75.0,...,2001-08-16,Very easy to make for a lunch. I served it co...,3,7.080531,4.477337,3.970292,4.234107,4.317488,5.030438,3.828641
4,10722,desserts,cherry cream cheese pie,6,1455,317.1,32.0,29.0,4.0,6.0,...,2001-09-04,I like easy and good. I bought the pie shell ...,5,5.759217,3.465736,3.367296,1.386294,1.791759,4.174387,2.197225


In [7]:
import math

date_80th = all_dataset.iloc[math.floor(.8 * len(all_dataset))]['date']
print('Date of the 80th-percentile review: ', date_80th)  

i_date = all_dataset.index[all_dataset['date'] == date_80th].tolist()
print(f'Index of the first review on {date_80th}: ', i_date[0]) 

Date of the 80th-percentile review:  2010-09-09
Index of the first review on 2010-09-09:  72257


In [8]:
train_set = all_dataset.iloc[:i_date[0]]
print(train_set.shape)
train_set.tail()

(72257, 23)


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
72252,349813,vegetables,healthier french fries and budget friendly,3,53,283.4,21.0,6.0,1.0,8.0,...,2010-09-08,This is a really nice basic fries recipe. I li...,4,5.646859,3.044522,1.791759,0.000000,2.079442,2.197225,2.484907
72253,76930,pasta,funky chicken with sesame noodles,11,210,900.8,35.0,147.0,160.0,97.0,...,2010-09-08,Fantastic! My boys devoured it! I used light s...,5,6.803283,3.555348,4.990433,5.075174,4.574711,2.833213,3.713572
72254,214193,desserts,no egg tomato soup cake,12,75,348.4,18.0,118.0,22.0,8.0,...,2010-09-08,"Oh wow, this is a really, really good cake rec...",5,5.853351,2.890372,4.770685,3.091042,2.079442,3.610918,2.890372
72255,177792,vegetables,chili potato dippers with cheddar jalapeno dip,9,40,185.9,15.0,8.0,4.0,6.0,...,2010-09-08,"Mmm, yummy!!!! The fries are so spicy and yumm...",5,5.225209,2.708050,2.079442,1.386294,1.791759,2.639057,1.945910
72256,424718,vegetables,summer salad with goat cheese dutch,10,15,463.2,58.0,57.0,10.0,29.0,...,2010-09-08,Excellent repeater! Made dressing and nectari...,5,6.138159,4.060443,4.043051,2.302585,3.367296,4.127134,1.791759


In [9]:
test_set = all_dataset.iloc[i_date[0]:]
print(test_set.shape)
test_set.head()

(18076, 23)


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
72257,436255,beverages,gingered watermelon juice,2,10,78.2,1.0,63.0,1.0,3.0,...,2010-09-09,Very refreshing on a warm day of Indian summer...,5,4.359270,0.000000,4.143135,0.000000,1.098612,0.000000,1.791759
72258,426940,pasta,pastitso macaroni and meat,15,90,1090.8,108.0,15.0,82.0,108.0,...,2010-09-09,"Well, LIG, after I dirtied every pot and pan i...",5,6.994667,4.682131,2.708050,4.406719,4.682131,5.214936,2.944439
72259,77206,vegetables,almost boston market creamed spinach,10,50,337.9,43.0,9.0,24.0,18.0,...,2010-09-09,"I don't know about Boston Market, Kitten, but ...",5,5.822750,3.761200,2.197225,3.178054,2.890372,4.430817,1.609438
72260,336781,vegetables,beef stir fry asian style,12,35,283.2,26.0,8.0,14.0,46.0,...,2010-09-09,"Quick, easy and delicious. I marinated the be...",5,5.646153,3.258097,2.079442,2.639057,3.828641,3.295837,1.098612
72261,178213,seafood,garlic prawns in the microwave,7,35,257.0,29.0,1.0,29.0,34.0,...,2010-09-09,Divine sauce-the sherry is wonderful in this! ...,5,5.549076,3.367296,0.000000,3.367296,3.526361,2.708050,0.000000


In [11]:
# convert to csv
train_set.to_csv('./Dataset/train_dataset_all.csv', index=False)
test_set.to_csv('./Dataset/test_dataset_all.csv', index=False)

## Test-validation dataset

In [12]:
val_set = test_set[:len(test_set)//2]
val_set

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
72257,436255,beverages,gingered watermelon juice,2,10,78.2,1.0,63.0,1.0,3.0,...,2010-09-09,Very refreshing on a warm day of Indian summer...,5,4.359270,0.000000,4.143135,0.000000,1.098612,0.000000,1.791759
72258,426940,pasta,pastitso macaroni and meat,15,90,1090.8,108.0,15.0,82.0,108.0,...,2010-09-09,"Well, LIG, after I dirtied every pot and pan i...",5,6.994667,4.682131,2.708050,4.406719,4.682131,5.214936,2.944439
72259,77206,vegetables,almost boston market creamed spinach,10,50,337.9,43.0,9.0,24.0,18.0,...,2010-09-09,"I don't know about Boston Market, Kitten, but ...",5,5.822750,3.761200,2.197225,3.178054,2.890372,4.430817,1.609438
72260,336781,vegetables,beef stir fry asian style,12,35,283.2,26.0,8.0,14.0,46.0,...,2010-09-09,"Quick, easy and delicious. I marinated the be...",5,5.646153,3.258097,2.079442,2.639057,3.828641,3.295837,1.098612
72261,178213,seafood,garlic prawns in the microwave,7,35,257.0,29.0,1.0,29.0,34.0,...,2010-09-09,Divine sauce-the sherry is wonderful in this! ...,5,5.549076,3.367296,0.000000,3.367296,3.526361,2.708050,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81290,5211,soup,five minute chicken vegetable soup,6,6,18.6,1.0,7.0,1.0,3.0,...,2012-04-28,Tasty soup that is quick & easy to prepare. I...,5,2.923162,0.000000,1.945910,0.000000,1.098612,0.000000,0.000000
81291,277737,meat,ham pineapple pizza muffins,6,30,214.1,11.0,44.0,28.0,26.0,...,2012-04-28,Super easy and a twist on an old favorite. Gro...,5,5.366443,2.397895,3.784190,3.332205,3.258097,2.944439,2.079442
81292,25551,pasta,hamburger skillet supper,4,15,336.5,31.0,1.0,21.0,46.0,...,2012-04-28,This was a hit at our house tonight. After a ...,4,5.818598,3.433987,0.000000,3.044522,3.828641,3.713572,1.386294
81293,115616,vegetables,parmesan asparagus and bell pepper,5,20,39.7,1.0,13.0,1.0,6.0,...,2012-04-29,I don't have a contact grill so I baked the ve...,5,3.681351,0.000000,2.564949,0.000000,1.791759,0.000000,0.693147


In [13]:
test_set2 = test_set[len(test_set)//2:]
test_set2

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
81295,460723,pasta,wheat berry salad,9,75,167.6,25.0,7.0,1.0,5.0,...,2012-04-29,"This is a very nice salad, one that can be the...",4,5.121580,3.218876,1.945910,0.000000,1.609438,2.197225,0.000000
81296,298977,vegetables,chicken salad tortillas,11,27,214.9,19.0,8.0,24.0,39.0,...,2012-04-29,"This made a super yummy, very filling lunch. L...",5,5.370173,2.944439,2.079442,3.178054,3.663562,2.302585,0.000000
81297,96160,pasta,oh so tasty rice,8,10,1178.4,123.0,24.0,75.0,90.0,...,2012-04-29,I agree this dish is very tasty. I was using ...,5,7.071913,4.812184,3.178054,4.317488,4.499810,4.934474,3.135494
81298,98500,pasta,spaghetti with italian sausage,8,30,723.9,39.0,84.0,64.0,51.0,...,2012-04-29,Great recipe to jazz up your regular spaghetti...,5,6.584653,3.663562,4.430817,4.158883,3.931826,3.637586,3.433987
81299,474623,meat,prosciutto wrapped stuffed chicken with herbed...,14,35,357.4,34.0,1.0,8.0,70.0,...,2012-04-29,Aussie Swap #63: AMAZING!! I love the chicken ...,5,5.878856,3.526361,0.000000,2.079442,4.248495,3.806662,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90328,289268,soup,mediterranean kale white bean soup with sausage,14,55,330.1,26.0,13.0,40.0,38.0,...,2018-12-02,I had to make small adjustments according to w...,5,5.799396,3.258097,2.564949,3.688879,3.637586,3.178054,2.197225
90329,140486,desserts,basic vanilla frosting,4,5,1162.8,37.0,940.0,9.0,2.0,...,2018-12-05,This was incredibly simple and easy to put tog...,5,7.058586,3.610918,6.845880,2.197225,0.693147,4.330733,4.382027
90330,121066,pasta,garlic shrimp pasta,10,30,408.1,14.0,8.0,20.0,41.0,...,2018-12-09,I doubled the garlic and felt that it could us...,4,6.011512,2.639057,2.079442,2.995732,3.713572,3.218876,2.944439
90331,19653,desserts,roll out cookies,6,25,1870.6,147.0,402.0,50.0,43.0,...,2018-12-10,These cookies were wonderful. I've had this in...,5,7.534015,4.990433,5.996452,3.912023,3.761200,5.690359,4.343805


In [14]:
# convert to csv
val_set.to_csv('./Dataset/validation_dataset_all.csv', index=False)
test_set2.to_csv('./Dataset/test_v2_dataset_all.csv', index=False)